In [30]:
# import face_recognition as fr
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import glob
import numpy as np
import re
import pandas as pd

In [31]:
paths =["data/CK+/","data/fer/train/"]
# data  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs = []
state = []

In [32]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
            state.extend([item for p in listdir(f"{path}{item}")])

In [33]:
len(state),len(imgs)

(29611, 29611)

In [34]:
imgs[:5],state[:5]

(['data/CK+/surprise/S052_001_00000015.png',
  'data/CK+/surprise/S050_002_00000018.png',
  'data/CK+/surprise/S086_001_00000019.png',
  'data/CK+/surprise/S037_001_00000020.png',
  'data/CK+/surprise/S034_001_00000029.png'],
 ['surprise', 'surprise', 'surprise', 'surprise', 'surprise'])

In [35]:
imgs_ = []
for p in imgs:
    temp = Image.open(p)
    save = temp.copy()
    imgs_.append(save)
    temp.close()


In [36]:
imgs_array = []
HEIGHT, WIDTH =75,75
for f in imgs_:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array.append(np.array(img))

In [37]:
#imgs_array = [el/255 for el in imgs_array]


In [38]:
imgs_array = np.array(imgs_array)

In [39]:
imgs_array.shape[1:]

(75, 75, 3)

In [40]:
categories = list(set(state))

In [41]:
categories

['anger', 'surprise', 'sadness', 'disgust', 'fear', 'neutral', 'happiness']

In [42]:
y=pd.DataFrame(state)
y_dummies = pd.get_dummies(y)

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs_array,y_dummies)

In [44]:
X_train.shape

(22208, 75, 75, 3)

In [45]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD, Adamax
from tensorflow.keras.applications import inception_v3

import keras

In [46]:
model_feats = inception_v3.InceptionV3(weights="imagenet", include_top=False, input_shape=(75,75,3))

In [47]:
model_feats.input_shape

(None, 75, 75, 3)

In [48]:
model_feats.output_shape

(None, 1, 1, 2048)

In [49]:
output = Sequential()
output.add(Flatten(input_shape=[1,1,2048]))
output.add(Dense(64,activation = 'relu'))
output.add(Dense(128,activation = 'relu'))
output.add(Dense(256,activation = 'relu'))
output.add(Dense(512,activation = 'relu'))
output.add(Dropout(0.3))
output.add(BatchNormalization())
output.add(Dense(7, activation='softmax'))

In [50]:
opt = Adam(learning_rate=0.001)

In [51]:
# model = Sequential()
# #model.add(Flatten())
# model.add(Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(imgs_array.shape[1:])))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Conv2D(filters=512, kernel_size=3, activation='relu', input_shape=(imgs_array.shape[1:])))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Flatten())
# model.add(Dense(64,activation = 'relu'))
# model.add(Dense(128,activation = 'relu'))
# model.add(Dense(256,activation = 'relu'))
# model.add(Dense(512,activation = 'relu'))
# model.add(Dropout(0.3))
# model.add(BatchNormalization())
# model.add(Dense(7, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])

In [52]:
model = Sequential([model_feats,output])
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])

In [53]:
from keras.models import Sequential, load_model, model_from_json
from keras import callbacks, optimizers
import tensorflow as tf
from datetime import date

fecha=str(date.today().year)+str(date.today().month)+str(date.today().day)    
symbol = 'faces_iception_2'
h5 = symbol + '_best_model' + fecha + '.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                       monitor='loss',
                                       verbose=0,
                                       save_best_only=True,
                                       #save_weights_only=True,
                                       mode='auto',
                                       save_freq=1)
callback = [checkpoint]
json = symbol + '_best_model' + fecha + '.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)

modelo = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 5000,callbacks = callback,validation_split = 0.1)

Epoch 1/5000
625/625 [==============================] - 763s 1s/step - loss: 1.7618 - accuracy: 0.3132 - val_loss: 2041.0928 - val_accuracy: 0.1247
Epoch 2/5000
625/625 [==============================] - 359s 575ms/step - loss: 1.6616 - accuracy: 0.3499 - val_loss: 1.7242 - val_accuracy: 0.3138
Epoch 3/5000
625/625 [==============================] - 196s 314ms/step - loss: 1.5759 - accuracy: 0.3835 - val_loss: 1.5327 - val_accuracy: 0.3908
Epoch 4/5000
625/625 [==============================] - 118s 189ms/step - loss: 1.4693 - accuracy: 0.4273 - val_loss: 1.6381 - val_accuracy: 0.4728
Epoch 5/5000
625/625 [==============================] - 39s 63ms/step - loss: 1.3818 - accuracy: 0.4658 - val_loss: 16.2984 - val_accuracy: 0.4093
Epoch 6/5000
625/625 [==============================] - 26s 41ms/step - loss: 1.3644 - accuracy: 0.4773 - val_loss: 1.5724 - val_accuracy: 0.4746
Epoch 7/5000
625/625 [==============================] - 25s 41ms/step - loss: 1.3793 - accuracy: 0.4671 - val_loss:

Epoch 57/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.1726 - accuracy: 0.9462 - val_loss: 2.0023 - val_accuracy: 0.5727
Epoch 58/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0950 - accuracy: 0.9731 - val_loss: 1.9636 - val_accuracy: 0.5777
Epoch 59/5000
625/625 [==============================] - 27s 44ms/step - loss: 0.1127 - accuracy: 0.9660 - val_loss: 2.0941 - val_accuracy: 0.5741
Epoch 60/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0884 - accuracy: 0.9760 - val_loss: 2.0662 - val_accuracy: 0.5849
Epoch 61/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.1086 - accuracy: 0.9678 - val_loss: 2.0409 - val_accuracy: 0.5817
Epoch 62/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.1085 - accuracy: 0.9678 - val_loss: 2.0864 - val_accuracy: 0.5682
Epoch 63/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0884 - accuracy: 0.9733 - val_loss: 2

625/625 [==============================] - 26s 42ms/step - loss: 0.0443 - accuracy: 0.9876 - val_loss: 2.0672 - val_accuracy: 0.5898
Epoch 113/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0248 - accuracy: 0.9930 - val_loss: 2.3485 - val_accuracy: 0.5907
Epoch 114/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0379 - accuracy: 0.9883 - val_loss: 2.0884 - val_accuracy: 0.5835
Epoch 115/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0506 - accuracy: 0.9843 - val_loss: 2.3589 - val_accuracy: 0.5907
Epoch 116/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0420 - accuracy: 0.9881 - val_loss: 2.1968 - val_accuracy: 0.6029
Epoch 117/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0290 - accuracy: 0.9919 - val_loss: 2.1878 - val_accuracy: 0.5912
Epoch 118/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0360 - accuracy: 0.9890 - val_loss: 2.2105 - 

625/625 [==============================] - 26s 42ms/step - loss: 0.0294 - accuracy: 0.9915 - val_loss: 2.1371 - val_accuracy: 0.6074
Epoch 168/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0204 - accuracy: 0.9930 - val_loss: 2.1893 - val_accuracy: 0.5849
Epoch 169/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0199 - accuracy: 0.9935 - val_loss: 2.2684 - val_accuracy: 0.6006
Epoch 170/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0278 - accuracy: 0.9926 - val_loss: 2.0016 - val_accuracy: 0.6060
Epoch 171/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0224 - accuracy: 0.9928 - val_loss: 2.2835 - val_accuracy: 0.5952
Epoch 172/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0272 - accuracy: 0.9918 - val_loss: 2.1607 - val_accuracy: 0.5988
Epoch 173/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0213 - accuracy: 0.9927 - val_loss: 2.2609 - 

625/625 [==============================] - 26s 41ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 2.3109 - val_accuracy: 0.6011
Epoch 223/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0142 - accuracy: 0.9951 - val_loss: 2.4370 - val_accuracy: 0.6078
Epoch 224/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0197 - accuracy: 0.9937 - val_loss: 2.3438 - val_accuracy: 0.5961
Epoch 225/5000
625/625 [==============================] - 26s 41ms/step - loss: 0.0169 - accuracy: 0.9953 - val_loss: 2.2397 - val_accuracy: 0.5952
Epoch 226/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0192 - accuracy: 0.9936 - val_loss: 2.2641 - val_accuracy: 0.5876
Epoch 227/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0201 - accuracy: 0.9929 - val_loss: 2.2814 - val_accuracy: 0.5889
Epoch 228/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0143 - accuracy: 0.9950 - val_loss: 2.3177 - 

625/625 [==============================] - 27s 43ms/step - loss: 0.0139 - accuracy: 0.9957 - val_loss: 2.3869 - val_accuracy: 0.5952
Epoch 278/5000
625/625 [==============================] - 27s 43ms/step - loss: 0.0142 - accuracy: 0.9956 - val_loss: 2.4074 - val_accuracy: 0.5975
Epoch 279/5000
625/625 [==============================] - 27s 43ms/step - loss: 0.0152 - accuracy: 0.9952 - val_loss: 2.4876 - val_accuracy: 0.5822
Epoch 280/5000
625/625 [==============================] - 27s 43ms/step - loss: 0.0177 - accuracy: 0.9941 - val_loss: 2.4580 - val_accuracy: 0.5925
Epoch 281/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0115 - accuracy: 0.9957 - val_loss: 2.4668 - val_accuracy: 0.6087
Epoch 282/5000
625/625 [==============================] - 27s 43ms/step - loss: 0.0193 - accuracy: 0.9938 - val_loss: 2.2999 - val_accuracy: 0.5943
Epoch 283/5000
625/625 [==============================] - 27s 43ms/step - loss: 0.0127 - accuracy: 0.9958 - val_loss: 2.3946 - 

625/625 [==============================] - 25s 39ms/step - loss: 0.0128 - accuracy: 0.9959 - val_loss: 2.3684 - val_accuracy: 0.5912
Epoch 333/5000
625/625 [==============================] - 25s 40ms/step - loss: 0.0068 - accuracy: 0.9971 - val_loss: 2.6645 - val_accuracy: 0.5939
Epoch 334/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0156 - accuracy: 0.9949 - val_loss: 2.5461 - val_accuracy: 0.5768
Epoch 335/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0125 - accuracy: 0.9957 - val_loss: 2.4596 - val_accuracy: 0.5961
Epoch 336/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0106 - accuracy: 0.9961 - val_loss: 2.6266 - val_accuracy: 0.5948
Epoch 337/5000
625/625 [==============================] - 26s 42ms/step - loss: 0.0136 - accuracy: 0.9954 - val_loss: 2.3277 - val_accuracy: 0.6029
Epoch 338/5000
625/625 [==============================] - 27s 43ms/step - loss: 0.0147 - accuracy: 0.9957 - val_loss: 2.3575 - 

KeyboardInterrupt: 

In [38]:
paths = ["data/predict/"] #["data/predict/","data/fer/test/"]
# data_pred  = listdir(path)
ignore = ["morralla",".DS_Store","contempt"]
imgs_pred = []
state_pred = []

In [39]:
for path in paths:
    for item in listdir(path):
        if item not in ignore:
            imgs_pred.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
            state_pred.extend([item for p in listdir(f"{path}{item}")])



In [40]:
imgs_pred

['data/predict/surprise/1636579058672.jpg',
 'data/predict/surprise/1636579058703.jpg',
 'data/predict/surprise/1636579058689.jpg',
 'data/predict/anger/1636579058658.jpg',
 'data/predict/anger/1636579058692.jpg',
 'data/predict/neutral/neutral_2.jpg',
 'data/predict/neutral/neutral_3.jpg',
 'data/predict/neutral/1636579058676.jpg',
 'data/predict/neutral/neutral_0.jpg',
 'data/predict/neutral/1636579058682.jpg',
 'data/predict/neutral/1636579058712.jpg',
 'data/predict/neutral/neutral_1.jpg',
 'data/predict/neutral/neutral_4.jpg',
 'data/predict/happiness/happy_6.png',
 'data/predict/happiness/happy_8.png',
 'data/predict/happiness/1636579058696.jpg',
 'data/predict/happiness/happy_4.png',
 'data/predict/happiness/happy_10.png',
 'data/predict/happiness/happy_11.png',
 'data/predict/happiness/1636579058663.jpg',
 'data/predict/happiness/1636579058717.jpg',
 'data/predict/happiness/happy_3.jpeg',
 'data/predict/happiness/happy_14.jpg',
 'data/predict/happiness/happy_9.png',
 'data/pred

In [41]:
imgs_pred_ = []
for p in imgs_pred:
    temp = Image.open(p)
    save = temp.copy()
    imgs_pred_.append(save)
    temp.close()

imgs_array_pred = []
for f in imgs_pred_:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array_pred.append(np.array(img))
imgs_array_pred = np.array(imgs_array_pred)

In [42]:
prediction = model.predict(imgs_array_pred)

In [43]:
len(prediction)

34

In [44]:
prediction[:5]

array([[6.1021964e-03, 8.7440317e-04, 9.7397953e-01, 1.0640242e-02,
        3.1060001e-03, 4.1390292e-04, 4.8838877e-03],
       [1.1344533e-02, 1.5153108e-03, 9.6891606e-01, 8.2021318e-03,
        5.0080214e-03, 6.8873167e-04, 4.3251230e-03],
       [2.8427225e-01, 2.6198642e-02, 7.9301288e-03, 1.3626334e-03,
        6.7581040e-01, 4.1654296e-03, 2.6052361e-04],
       [9.8083234e-01, 9.7697061e-03, 6.6696777e-04, 2.0539167e-04,
        7.8163873e-03, 6.1591150e-04, 9.3230170e-05],
       [1.0409561e-03, 9.5198286e-04, 7.9968839e-04, 3.5151860e-04,
        9.9646723e-01, 1.2970861e-04, 2.5880281e-04]], dtype=float32)

In [45]:
prediction.argmax(axis = -1)


array([2, 2, 4, 0, 4, 0, 4, 0, 1, 0, 2, 2, 4, 3, 3, 4, 0, 3, 3, 2, 4, 4,
       6, 3, 0, 3, 0, 2, 0, 4, 0, 1, 4, 4])

In [46]:
cats= y_dummies.columns
cats = [x.replace("0_","") for x in cats]

In [47]:
cats

['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

In [48]:
states_model = []
for i in range(len(prediction)):
    states_model.append(cats[prediction[i].argmax()])
    print(state_pred[i],cats[prediction[i].argmax()],imgs_pred[i])

surprise fear data/predict/surprise/1636579058672.jpg
surprise fear data/predict/surprise/1636579058703.jpg
surprise neutral data/predict/surprise/1636579058689.jpg
anger anger data/predict/anger/1636579058658.jpg
anger neutral data/predict/anger/1636579058692.jpg
neutral anger data/predict/neutral/neutral_2.jpg
neutral neutral data/predict/neutral/neutral_3.jpg
neutral anger data/predict/neutral/1636579058676.jpg
neutral disgust data/predict/neutral/neutral_0.jpg
neutral anger data/predict/neutral/1636579058682.jpg
neutral fear data/predict/neutral/1636579058712.jpg
neutral fear data/predict/neutral/neutral_1.jpg
neutral neutral data/predict/neutral/neutral_4.jpg
happiness happiness data/predict/happiness/happy_6.png
happiness happiness data/predict/happiness/happy_8.png
happiness neutral data/predict/happiness/1636579058696.jpg
happiness anger data/predict/happiness/happy_4.png
happiness happiness data/predict/happiness/happy_10.png
happiness happiness data/predict/happiness/happy_11

In [49]:
from sklearn.metrics import accuracy_score

In [50]:
accuracy_score(state_pred,states_model)

0.2647058823529412